# 准备
## 加载库函数及导入文件

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
import pathlib
from corelibs.config import *
warnings.filterwarnings('ignore', message="Workbook contains no default style, apply openpyxl's default", category=UserWarning)
conf_data = load_conf()
OUTPUT_DIR_PATH = pathlib.Path(r"J:\银行分析结果")
OUTPUT_DIR_PATH = pathlib.Path(r"D:\gitcodes\testdata\银行分析结果test")
OUTPUT_DIR_PATH = pathlib.Path(r"/mnt/d/gitcodes/testdata/银行分析结果test/")
OUTPUT_DIR_PATH

PosixPath('/mnt/d/gitcodes/testdata/银行分析结果test')

## 重新加载配置

In [ ]:
load_conf()
conf_obj = get_conf_obj('央地协查','客户')
[print(n,v) for n,v in zip(conf_obj._fields, conf_obj)]

## 提取excel表头和md5值

In [18]:
from hashlib import md5
from corelibs.header import read_header
acc_file = pathlib.Path(r"/mnt/d/gitcodes/testdata/000615/王瑞霞_410527198607032042/王瑞霞]招商银行-交易流水-交易流水记录.xlsx")
print(a := read_header(acc_file, header=0))
md5(a.encode()).hexdigest()

('日志号', '交易时间', '交易对方名称', '交易余额', '查询卡号', '商户名称', '交易流水号', '商户号', '交易发生地', '交易是否成功', '币种', '凭证种类', '交易柜员号', '终端号', '交易对方卡号', '现金标志', '交易对手余额', 'IP地址', '交易网点名称', '交易对方证件号码', '备注', '交易摘要', '本方账号', '交易对方账号开户行', 'MAC地址', '借贷标志', '交易网点代码', '交易对方账号', '交易类型', '传票号', '交易金额', '查询账号', '本方卡号', '凭证号')


'69285347f3af24531fbca71e25743fb2'

In [ ]:
conf_data

## 识别文件列表中的文件类型并进行数据验证

In [9]:
from collections import Counter
from corelibs.process import classify_files_by_category
base_dir = pathlib.Path(r"/mnt/d/gitcodes/testdata/000615/王瑞霞_410527198607032042/")
_file_names = list(filter(lambda f: not f.stem.endswith('关联子账户信息'), base_dir.glob('*光大*.xlsx')))
f, e = classify_files_by_category(_file_names)
print(f'以下是识别错误文件：')
len([print(k.name + v) for k, v in e.items()])

识别文件类型:   0%|          | 0/3 [00:00<?, ?it/s]

光大银行-账户信息-客户基本信息.xlsx => 央地协查:客户
光大银行-账户信息-账户基本信息.xlsx => 央地协查:账户
王瑞霞]光大银行-交易流水-交易流水记录.xlsx => 央地协查:流水
以下是识别错误文件：


0

# 执行
## 分析建设银行网点数据目录
（如：开发区金融街建行查询数据）

### 新版分析，可以根据账户信息更新账号数据：

In [ ]:
from banks.ccb import process_dir_ccb_branch_v2
base_dir = pathlib.Path(r"/mnt/d/gitcodes/testdata/建行")
doc_No = 'aaa'
re = process_dir_ccb_branch_v2(base_dir, OUTPUT_DIR_PATH, doc_No)

### 老版分析，不能根据账户信息更新账号数据：

In [ ]:
# from banks.ccb import process_dir_ccb_branch_v1
# base_dir = pathlib.Path(r"/mnt/d/gitcodes/testdata/建行")
# doc_No = 'aaa'
# err_files = process_dir_ccb_branch_v1(base_dir, OUTPUT_DIR_PATH, doc_No)

## 分析央地协查数据目录

In [11]:
from banks.yangdi import process_dir_yangdi
base_dir = pathlib.Path(r"/mnt/d/gitcodes/testdata/000615/王瑞霞_410527198607032042/")
doc_No = 'aaa'
re = process_dir_yangdi(base_dir, OUTPUT_DIR_PATH, doc_No)

识别文件类型:   0%|          | 0/3 [00:00<?, ?it/s]

广发银行-账户信息-客户基本信息.xlsx => 央地协查:客户
广发银行-账户信息-账户基本信息.xlsx => 央地协查:账户
广发银行-金融理财-账户基本信息.xlsx => 未找到对应配置，跳过
1个文件未识别：[Y继续/非Y显示详情并退出]


广发银行-金融理财-账户基本信息.xlsx => 未找到对应配置，跳过


TypeError: cannot unpack non-iterable NoneType object

## 继续处理出错文件

In [ ]:
from tqdm.auto import tqdm
from corelibs.process import process_files_accs_then_stats
df_acc = re[0]
err_files: dict = re[1]
re = process_files_accs_then_stats(err_files.keys(), OUTPUT_DIR_PATH, doc_No, df_acc)

# 测试

## 测试yaml配置文件语法，及内部操作指令对象转换

In [7]:
load_conf()
print(get_output_format())
print(len([print(x) for x in get_header_hash().items()]))

excel
('6d3d0d1e252f43a000010e19d8b2dd57', ['天津银行', '账户'])
('7509cd85808831c7004c8c78e9267d7c', ['天津银行', '流水'])
('5b70f468037b3cfe76bace72751f4e3e', ['工商银行网点', '账户'])
('0b02a413bd3c59d74a82e3fd3909c759', ['工商银行网点', '流水'])
('5ff3970075646f3a7fffdef5430539d7', ['建设银行网点', '账户'])
('8fa9c0776b857793f91335192398aeba', ['建设银行网点', '流水', '交易'])
6


In [ ]:
print(len([print(x) for x in get_conf_cache().items()]))

##  测试单个非流水文件读取及储存

In [15]:
from corelibs.process import *
acc_file = pathlib.Path(r"/mnt/d/gitcodes/testdata/建行/1080860401675667712130255_5_1_watermark.xlsx")
acc_file = pathlib.Path(r"/mnt/d/gitcodes/testdata/000615/王瑞霞_410527198607032042/光大银行-账户信息-客户基本信息.xlsx")
print(b := get_file_type(acc_file))
df_acc = process_general_file(acc_file, OUTPUT_DIR_PATH, b[0], '客户', '1')

['央地协查', '客户']
     银行     姓名                证件号码            电话   地址 工作单位         联系手机 住宅电话  \
0  光大银行    王瑞霞  410527198607032042  13752336375   NaN  NaN  13752336375  NaN   
1  光大银行  hahah               11111           NaN  NaN  NaN          NaN  NaN   
0  光大银行    王瑞霞  410527198607032042  13752336375   NaN  NaN  13752336375  NaN   
1  光大银行  hahah               11111           NaN  NaN  NaN          NaN  NaN   

  联系电话 单位电话  ... 法人代表证件号码 法人代表证件类型 邮箱地址 地税纳税号 国税纳税号 客户工商执照号码 代办人姓名 代办人证件号码  \
0  NaN  NaN  ...      NaN      NaN  NaN   NaN   NaN      NaN   NaN     NaN   
1  NaN  NaN  ...      NaN      NaN  NaN   NaN   NaN      NaN   NaN     NaN   
0  NaN  NaN  ...      NaN      NaN  NaN   NaN   NaN      NaN   NaN     NaN   
1  NaN  NaN  ...      NaN      NaN  NaN   NaN   NaN      NaN   NaN     NaN   

  代办人证件类型  证照类型代码  
0     NaN  110001  
1     NaN     NaN  
0     NaN  110001  
1     NaN     NaN  

[4 rows x 24 columns]


In [ ]:
df_acc

In [ ]:
df_acc.drop_duplicates()

In [ ]:
import dtale
dtale.show(df_acc, open_browser=True)

## 测试单个流水文件的读取与储存

In [43]:
from corelibs.process import process_statment_file_general
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_2_1_watermark.xlsx")
# acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217号.xls")
# acc_file = pathlib.Path(r"/mnt/d/gitcodes/testdata/建行(于东林、刘德阳等)【已分析】/1080860401675667712130255_6_1_watermark.xlsx")
df = process_statment_file_general(acc_file, OUTPUT_DIR_PATH,'建设银行网点', '流水','a', df_acc)

In [ ]:
import dtale
dtale.show(df, open_browser=True)

## 测试文件夹中部分文件

In [ ]:
base_dir = pathlib.Path(r'D:\gitcodes\testdata\建行')
list(base_dir.glob('[!~]*.xlsx')) 


[]

In [45]:
from corelibs.process import process_files_accs_then_stats
base_dir = pathlib.Path(r"/mnt/d/gitcodes/testdata/000615/王瑞霞_410527198607032042/")
doc_No = 'aaa'
_file_names = list(base_dir.glob('*光大*.xlsx')) # 找到目录中所有的excel文件（不含子目录）
process_files_accs_then_stats(_file_names, OUTPUT_DIR_PATH, doc_No)

识别文件类型:   0%|          | 0/4 [00:00<?, ?it/s]

光大银行-账户信息-关联子账户信息.xlsx => 未找到对应配置，跳过
光大银行-账户信息-客户基本信息.xlsx => 央地协查:客户
光大银行-账户信息-账户基本信息.xlsx => 央地协查:账户
王瑞霞]光大银行-交易流水-交易流水记录.xlsx => 央地协查:流水
以下为发生错误文件：
光大银行-账户信息-关联子账户信息.xlsx => 未找到对应配置，跳过


# 杂项

In [ ]:
import pandas as pd
df2 = pd.DataFrame({'a':[1,2,3,4],'b':[5,6,7,8],'c':[9,10,11,12]})
df1 = pd.DataFrame({'a':[1,2,1,5],'b':[1,2,1,5],'d':[13,14,15,16]})
print(df1)
print(df2)
a = pd.merge(df1,df2,left_on='a',right_on='a', how='left', validate='m:1', copy=False)
a['c'].where(a['c'].notnull(), a['d'], inplace=True)
a

In [8]:
[1,2,3,4,5,6,7] + [8]

[1, 2, 3, 4, 5, 6, 7, 8]

In [9]:
import pandas as pd
import numpy as np
pd.concat([None, None])

ValueError: All objects passed were None